# 連接資料庫並取得評論資料

In [82]:
import os
import pandas as pd
from sqlalchemy import create_engine, text, bindparam

In [83]:
db_user = 'root'
db_password = '123456'
db_host = '100.77.42.49'
db_port = '3306'
db_name = 'lda_temp'

# 建立連線
connection_string = f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}?charset=utf8mb4"
engine = create_engine(connection_string)

try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT 1"))
        print("連線成功！")
except Exception as e:
    print(f"連線失敗：{e}")

連線成功！


In [84]:
sql_file_path = 'sql_query/test.sql'
sql_file_name = os.path.splitext(os.path.basename(sql_file_path))[0]

In [85]:
try:
    # 讀取 SQL 檔案
    if not os.path.exists(sql_file_path):
        raise FileNotFoundError(f"找不到檔案：{sql_file_path}")

    with open(sql_file_path, 'r', encoding='utf-8-sig') as file:
        sql_query = file.read()

    # 執行查詢並放入 dataFrame
    with engine.connect() as conn:
        df = pd.read_sql(text(sql_query), conn)

    print(f"讀取成功！共取得 {len(df)} 筆資料")
    # print(df.head())

except FileNotFoundError as e:
    print(e)
except Exception as e:
    print(f"發生錯誤: {e}")

讀取成功！共取得 1986 筆資料


# 分詞

In [86]:
import pandas as pd
from gensim import corpora, models
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim_models
import re
import os
import torch
import csv
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker
import tqdm as notebook_tqdm
from tqdm.auto import tqdm

In [87]:
OUTPUT_PATH = f"result_LDA"
STOP_WORDS_PATH = r"stop_dic/stopwords.txt"
CUSTOM_DICT_PATH = r'stop_dic/dict.txt'
POS_MAPPING_CSV_PATH = r"stop_dic/CKIP_Tag_Mapping_Table.csv"
DATA_PATH = r'data'
DEVICE = 0 if torch.cuda.is_available() else -1
print(f"{torch.cuda.is_available()}")

True


In [88]:
class CKIPTokenizer:
    """
    CKIPTokenizer 類別用於整合中研院 CKIP Transformers 斷詞工具
    """
    def __init__(self, stopwords_path, pos_mapping_csv_path, custom_dict_path=None, device=0):
        """
        初始化斷詞器與 CKIP 模型。
        :param stopwords_path: 停用詞檔案路徑 (.txt)
        :param pos_mapping_csv_path: 詞性對照表路徑 (.csv)，用於決定保留哪些詞性
        :param custom_dict_path: 自定義詞典路徑 (.txt)
        :param device: 執行設備 (0 為 GPU, -1 為 CPU)
        """
        self.stopwords = self.load_stopwords(stopwords_path)
        self.custom_dict = self.load_custom_dict(custom_dict_path) if custom_dict_path else {}
        self.pos_tags_to_keep = self.load_pos_tags_to_keep(pos_mapping_csv_path)
        
        # 初始化 CKIP 三大核心驅動器
        self.ws_driver  = CkipWordSegmenter (model="albert-base", device=device) # 斷詞
        self.pos_driver = CkipPosTagger     (model="albert-base", device=device) # 詞性標記
        self.ner_driver = CkipNerChunker    (model="albert-base", device=device) # 實體辨識
        
    def load_stopwords(self, filepath):
        """載入停用詞檔案，回傳一個 Set 以提升搜尋效率"""
        with open(filepath, encoding='utf-8-sig') as f:
            return set(line.strip() for line in f)

    def load_custom_dict(self, filepath):
        """載入自定義詞典，格式預期為：詞彙 [權重]"""
        custom_terms = {}
        with open(filepath, encoding='utf-8-sig') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 1:
                    word = parts[0]
                    # 若無指定權重，預設為 1.0
                    weight = float(parts[1]) if len(parts) > 1 else 1.0
                    custom_terms[word] = weight
        return custom_terms

    def load_pos_tags_to_keep(self, csv_path):
        """從 CSV 載入需要保留的詞性標記"""
        pos_tags = []
        with open(csv_path, newline='', encoding='utf-8-sig') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                # 檢查 CSV 中 '需要(0/1)' 欄位是否為 '1'
                if row['需要(0/1)'].strip() == '1':
                    tags = row['簡化標記']
                    pos_tags.append(tags)
        return pos_tags

    def tokenize(self, text, return_pos=False):
        """
        對單一字串進行斷詞
        :param text: 輸入純文字
        :param return_pos: 是否回傳 (詞, 詞性) 元組清單
        """
        if not isinstance(text, str):
            return ""

        # 清洗文字：僅保留中文 (Regex: \u4e00-\u9fff)
        text = re.sub(r'[^\u4e00-\u9fff]', '', text)
        if not text:
            return ""

        try:
            # 執行斷詞與詞性標記
            word_sentence_list = self.ws_driver([text], show_progress=False)
            pos_sentence_list = self.pos_driver(word_sentence_list, show_progress=False)

            words = word_sentence_list[0]
            pos_tags = pos_sentence_list[0]

            filtered_words = []
            filtered_word_pos = []
            
            # 過濾邏輯：1.不在停用詞內 2.字數>=2 3.屬於保留詞性
            for word, pos in zip(words, pos_tags):
                if (word not in self.stopwords) and (len(word) >= 2) and (pos in self.pos_tags_to_keep):
                    filtered_words.append(f"{word}")
                    filtered_word_pos.append((word, pos))

            if return_pos:
                return filtered_word_pos
            else:
                return " ".join(filtered_words) # 回傳以空格分隔的字串
        except Exception as e:
            print(f"CKIP processing error: {e}")
            return "" if not return_pos else []
        
    def batch_tokenize(self, text_list, batch_size=64):
        """
        批次處理多筆文字
        :param text_list: 字串列表
        :param batch_size: 批次處理大小
        """
        clean_texts = []
        valid_indices = [] # 用於記錄非空值的位置，以便最後還原清單長度
        
        print("正在清理文字...")
        for idx, text in enumerate(text_list):
            if isinstance(text, str):
                # 這裡保留了中英數字 ( \w )
                text = re.sub(r'[^\u4e00-\u9fff\w]', '', text) 
                if text.strip():
                    clean_texts.append(text)
                    valid_indices.append(idx)
        
        if not clean_texts:
            return [[] for _ in range(len(text_list))]

        # 批次執行模型運算
        print(f"執行 CKIP 模型 (Batch Size: {batch_size})...")
        ws_list = self.ws_driver(clean_texts, batch_size=batch_size, show_progress=True)
        pos_list = self.pos_driver(ws_list, batch_size=batch_size, show_progress=True)

        print("過濾停用詞與篩選詞性...")
        final_tokens_list = []
        
        for words, pos_tags in zip(ws_list, pos_list):
            filtered_words = []
            for word, pos in zip(words, pos_tags):
                # 條件篩選：非停用詞、長度 > 1、指定詞性
                if (word not in self.stopwords) and (len(word) > 1) and (pos in self.pos_tags_to_keep):
                    filtered_words.append(word)
            
            final_tokens_list.append(filtered_words)

        # 依照原始 list 的索引將結果放回對應位置，其餘填空 list
        result = [[] for _ in range(len(text_list))]
        for i, tokens in zip(valid_indices, final_tokens_list):
            result[i] = tokens
            
        return result

In [89]:
tokenizer = CKIPTokenizer(
    stopwords_path=STOP_WORDS_PATH, 
    pos_mapping_csv_path=POS_MAPPING_CSV_PATH, 
    custom_dict_path=CUSTOM_DICT_PATH, 
    device=DEVICE
)

In [90]:
print("處理 DataFrame...")
df['tokens'] = tokenizer.batch_tokenize(df['評論'].tolist(), batch_size=256)
print(df[['評論', 'tokens']].head())

處理 DataFrame...
正在清理文字...
執行 CKIP 模型 (Batch Size: 256)...


Inference: 100%|██████████| 8/8 [00:05<00:00,  1.49it/s]


過濾停用詞與篩選詞性...
                                                  評論  \
0  我的頭髮很多，而且不是那種很容易出油的髮質\n所以基本上不太挑洗髮精\n但不知道是不是今年天...   
1  去年9月所採購的洗髮品們，陸陸續續的使用，也即將用光光了～\n（因為＂不會＂再回購，所以在這...   
2  跟姊妹一起觀望一陣子，終於一起購入AVEDA洗髮精\n以前對洗髮這塊比較沒那麼重視\n總是以...   
3  每次洗完一瓶洗髮精都會換品牌，\n這個牌子是因為聽了podcast節目而認識的品牌，\n雖然...   
4  最近半年對頭髮的狀態實在是有點困擾\n本來就是細軟髮的髮質\n又因為冬季防冷戴各式帽子\n脫...   

                                              tokens  
0  [髮質, 基本, 今年, 天氣, 關係, 用到, 適合, 近期, 出門, 上班, 出遊, 戴...  
1  [去年, 9月, 採購, 洗髮品, 陸陸續續, 即將, 用光光, 回購, 空位, 用過, 淨...  
2  [姊妹, 一起, 觀望, 一陣子, 終於, 一起, 購入, 那麼, 重視, 總是, 香味, ...  
3  [牌子, 節目, 認識, 藥妝店, 當時, 看起來, 好像, 不錯, 晚上, 痛苦, 頭皮,...  
4  [最近, 狀態, 實在, 困擾, 本來, 軟髮, 髮質, 冬季, 防冷, 帽子, 脫下, 處...  


In [91]:
file_name = f"result_tokens.csv"
save_path = os.path.join(OUTPUT_PATH, file_name)

print(f"\n準備儲存檔案至: {save_path}")

try:
    # 檢查目錄是否存在，不存在就建立
    if not os.path.exists(OUTPUT_PATH):
        os.makedirs(OUTPUT_PATH)
        print(f"已建立資料夾: {OUTPUT_PATH}")

    df[['評論', 'tokens']].to_csv(
        save_path, 
        index=False, 
        encoding='utf-8-sig'
    )
    
    print(f"成功保存，檔案大小約: {os.path.getsize(save_path) / 1024:.2f} KB")

except Exception as e:
    print(f"保存失敗: {e}")


準備儲存檔案至: result_LDA/result_tokens.csv
成功保存，檔案大小約: 2031.53 KB


# LDA

## 建立詞頻矩陣

In [137]:
import pandas as pd
import ast
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [138]:
corpus = df['tokens'].apply(lambda x: ' '.join(x))
print(corpus.head(3))

0    髮質 基本 今年 天氣 關係 用到 適合 近期 出門 上班 出遊 戴完 安全帽 困擾 巴黎 ...
1    去年 9月 採購 洗髮品 陸陸續續 即將 用光光 回購 空位 用過 淨髮 精華露 感受 古寶...
2    姊妹 一起 觀望 一陣子 終於 一起 購入 那麼 重視 總是 香味 選擇 洗髮品 懷疑 高價...
Name: tokens, dtype: object


In [139]:
n_features = 1000  # 只取前 1000 個最重要的詞

# 建立 Vectorizer
tf_vectorizer = CountVectorizer(
    stop_words=list(tokenizer.stopwords),
    max_features=n_features,
    # max_df=0.70,                        # 如果一個詞在 95% 的文章都出現，就刪掉 (太通用)
    min_df=50,                           # 至少要出現 5 次才算數 (過濾錯字或極罕見詞)
    token_pattern=r'(?u)\b\w+\b'        # 確保匹配中文
)

# 轉換為矩陣 (詞袋（Bag-of-Words）)
tf = tf_vectorizer.fit_transform(corpus)

print(f"矩陣形狀: {tf.shape} (評論數, 特徵詞數)")
feature_names = tf_vectorizer.get_feature_names_out() # 詞彙表
print(f"前 50 個特徵詞: {feature_names[:50]}")

矩陣形狀: (1986, 275) (評論數, 特徵詞數)
前 50 個特徵詞: ['一定' '一起' '一陣子' '一點' '下午' '不同' '不用' '不能' '不錯' '中性' '主要' '乾性' '乾淨' '乾澀'
 '乾燥' '介紹' '以前' '便宜' '保濕' '保養' '修護' '健康' '價格' '光澤' '入手' '其實' '冬天' '出現'
 '刺激' '功效' '包裝' '卡詩' '印象' '原本' '去年' '受損' '只是' '只有' '可能' '台灣' '同時' '含有'
 '吹乾' '吹完' '味道' '嘗試' '嚴重' '回購' '困擾' '夏天']


In [140]:
# 算出每個詞在所有文章中的總次數
# tf 是一個稀疏矩陣 (sparse matrix)，sum() 出來會是一個矩陣物件
# .A1 是 numpy 的屬性，可以快速將矩陣攤平變成一維陣列 (array)
total_counts = tf.sum(axis=0).A1

# 建立 DataFrame 對照表 (詞, 次數)
word_freq_df = pd.DataFrame({
    'Term': feature_names,
    'Frequency': total_counts
})

# 依照次數由大到小排序
word_freq_df = word_freq_df.sort_values(by='Frequency', ascending=False)
print("\n=== 前 20 個高頻詞 ===")
print(word_freq_df.head(20))


=== 前 20 個高頻詞 ===
    Term  Frequency
256   頭皮       2979
44    味道       1118
263   香味        672
273   髮質        554
231   起泡        543
213   蓬鬆        505
236   透明        457
13    乾澀        448
135   洗頭        442
93    控油        422
149   清爽        418
228   質地        407
132   泡泡        389
272   髮絲        370
242   適合        363
8     不錯        355
12    乾淨        332
206   舒服        323
229   購入        313
128   油性        310


## 訓練 LDA 模型

In [141]:
# 設定主題數量
n_topics = 5

print(f"開始訓練 LDA 模型 (主題數: {n_topics})...")

lda = LatentDirichletAllocation(n_components=n_topics,                  # 主題數量
                                max_iter=200,                           # 疊代次數
                                learning_method='online',
                                learning_offset=50,
                                doc_topic_prior=0.000001,                   # 預設為 1 / n_topics，主題分布的稀疏程度（alpha）   (1 / n_topics)
                                topic_word_prior=0.00001,               # 預設為 1 / n_features，詞分布的稀疏程度（Beta）    (1 / n_features)
                                random_state=0)

lda.fit(tf)
print("模型訓練完成")

開始訓練 LDA 模型 (主題數: 5)...
模型訓練完成


## 查看主題關鍵字

In [142]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"主題 {topic_idx + 1}:")
        # 取得排序前 no_top_words 個關鍵字
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))
        print("-" * 20)

# 執行顯示：每個主題只顯示 10 個詞
display_topics(lda, tf_vectorizer.get_feature_names_out(), 20)

主題 1:
頭皮 頭皮屑 洗頭 掉髮 油性 狀況 隔天 適合 發現 改善 髮質 推薦 目前 落髮 夏天 已經 其實 不錯 購入 困擾
--------------------
主題 2:
頭皮 成分 清潔 清爽 控油 淨化液 添加 溫和 平衡 淨化 萃取 舒緩 深層 矽靈 按摩 介紹 髮絲 涼感 髮質 天然
--------------------
主題 3:
髮質 髮絲 頭皮 蓬鬆 透明 修護 柔順 質地 泡沫 起泡 毛躁 香味 護色 髮尾 髮根 護髮 香氣 扁塌 染髮 打結
--------------------
主題 4:
味道 香味 頭皮 起泡 泡泡 乾澀 透明 控油 舒服 蓬鬆 不錯 乾淨 質地 清爽 沖洗 淡淡 洗頭 回購 護髮 購入
--------------------
主題 5:
包裝 味道 容量 顏色 強健 養髮 草本 瓶身 當時 豐盈 藥妝店 設計 不同 質地 外觀 購買 髮根 價格 好像 購入
--------------------


In [143]:
from collections import defaultdict

def analyze_topic_overlap(model, feature_names, n_top_words):
    # 1. 取得每個主題的前 N 個關鍵詞
    topic_words = []
    for topic in model.components_:
        top_indices = topic.argsort()[:-n_top_words - 1:-1]
        words = [feature_names[i] for i in top_indices]
        topic_words.append(words)

    # 2. 統計每個詞出現在哪些主題中
    word_counts = defaultdict(list)
    for topic_idx, words in enumerate(topic_words):
        for word in words:
            word_counts[word].append(topic_idx + 1)

    # 3. 篩選出重複出現（出現在 2 個以上主題）的詞
    duplicates = {word: idxs for word, idxs in word_counts.items() if len(idxs) > 1}

    # 4. 顯示結果
    print("=== 跨主題重複關鍵詞統計 ===")
    if not duplicates:
        print("沒有發現重複的關鍵詞。")
    else:
        # 按照重複次數從多到少排序
        sorted_duplicates = sorted(duplicates.items(), key=lambda x: len(x[1]), reverse=True)
        
        print(f"{'關鍵詞':<10} | {'出現次數':<6} | {'所屬主題'}")
        print("-" * 40)
        for word, idxs in sorted_duplicates:
            idx_str = ", ".join([f"#{i}" for i in idxs])
            print(f"{word:<12} | {len(idxs):<8} | {idx_str}")
            
    return duplicates

# 執行統計分析
duplicate_dict = analyze_topic_overlap(lda, feature_names, n_top_words=15)

=== 跨主題重複關鍵詞統計 ===
關鍵詞        | 出現次數   | 所屬主題
----------------------------------------
頭皮           | 4        | #1, #2, #3, #4
質地           | 3        | #3, #4, #5
髮質           | 2        | #1, #3
清爽           | 2        | #2, #4
控油           | 2        | #2, #4
蓬鬆           | 2        | #3, #4
透明           | 2        | #3, #4
起泡           | 2        | #3, #4
香味           | 2        | #3, #4
味道           | 2        | #4, #5


## 輸出每個文章對應的主題

In [144]:
import numpy as np
topics=lda.transform(tf)
topic = []
for t in topics:
    topic.append(list(t).index(np.max(t)))
df['topic'] = topic
df.to_csv(f"{OUTPUT_PATH}/data_topic.csv",index = False, encoding='utf-8-sig')
# topics[0] #0 1 2 

## web可視化

In [145]:
import pyLDAvis
print(pyLDAvis.__version__)
import pyLDAvis.lda_model

3.4.0


In [146]:
# 在 Jupyter Notebook 中，直接顯示
pyLDAvis.enable_notebook()
vis = pyLDAvis.lda_model.prepare(lda, tf, tf_vectorizer)
pyLDAvis.display(vis)

In [147]:
# 保存成 HTML 檔案
html_path = os.path.join(OUTPUT_PATH, 'result_lda.html')
pyLDAvis.save_html(vis, html_path)

# 文本相似度

In [ ]:
import re
import jieba
import numpy as np
import tqdm as notebook_tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [ ]:
STOP_WORDS_PATH = r"stop_dic/stopwords.txt"

In [ ]:
# 專利摘要文本1，申請號：CN202480029985.2
patent1 = "本公開涉及通信方法、通信設備、通信系統及存儲介質。通信方法包括：AF向第一網元發送第一資訊，所述第一資訊用於指示取消A‑IoT設備的週期性操作。本公開通過AF向第一網元發送第一資訊，以指示取消A‑IoT設備的週期性操作，從而有效減少A‑IoT設備的能量消耗。"

# 專利摘要文本2，申請號：CN202480000774.6
patent2 = "本公開涉及通信方法、通信設備、通信系統及存儲介質。該方法包括：第一實體接收網路功能服務消費方發送的第一請求，所述第一請求包括網路功能服務提供方相關的第一資訊，所述第一資訊是基於所述網路功能服務提供方的第二資訊進行處理得到的資訊，所述網路功能服務消費方位於第一網路，所述網路功能服務提供方位於第二網路；所述第一實體發送第二請求。通過本公開實施例，可以提高通信安全性。"

# 專利摘要文本3，申請號：CN202410796819.7
patent3 = "本公開涉及一種螢幕的顯示方法、裝置、設備、存儲介質和程式產品，涉及螢幕顯示技術領域，方法包括：獲取終端生成的待顯示資料包，待顯示資料包包括目標幀對應的幀同步信號、目標幀中每個圖元行對應的行同步信號以及目標顯示資料；確定終端的螢幕對應的面板時序；根據面板時序，對行同步信號進行補償，得到目標行同步信號；根據幀同步信號、目標行同步信號和目標顯示資料，生成目標顯示信號，並在終端的螢幕上進行顯示。這樣，根據終端螢幕對應的面板時序對待顯示資料包中的行同步信號進行補償，從而避免了螢幕出現花屏、橫屏、亮帶等等異常情況，保證了螢幕的顯示效果和穩定性，提升了用戶體驗。"

## 1. 傳統方法_TF-IDF
Ref: https://doi.org/10.1111/jofi.12885

In [ ]:
# 1. 加載停用詞
def load_stopwords(filepath):
    with open(filepath, encoding='utf-8-sig') as f:
        return set(line.strip() for line in f)

STOPWORDS = load_stopwords(STOP_WORDS_PATH)

In [ ]:
# 2. jieba 分詞
def segment_sentence(sentence: str) -> list:
    words = jieba.cut(sentence, cut_all=False)
    return [w.strip() for w in words if w.strip()]

In [ ]:
# 3. 去除停用詞
def remove_stopwords(words: list) -> list:
    return [w for w in words if w not in STOPWORDS]

In [ ]:
# 4. 去除標點符號
def remove_punctuation(words: list) -> str:
    joined = " ".join(words)
    cleaned = re.sub(r"[^0-9A-Za-z\u4e00-\u9fa5\s]", " ", joined)
    cleaned = re.sub(r"\s+", " ", cleaned).strip()
    return cleaned

In [ ]:
# 5. 總流程
def preprocess_document(text: str) -> list:
    # Step1 分詞
    words = segment_sentence(text)

    # Step2 去停用詞
    words = remove_stopwords(words)

    # Step3 去標點符號
    cleaned_list = remove_punctuation(words)

    return cleaned_list

In [ ]:
text1 = preprocess_document(patent1)
text2 = preprocess_document(patent2)
text3 = preprocess_document(patent3)

In [ ]:
text1

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_tfidf_cosine_similarity(text1_words, text2_words):
    """
    使用 TF-IDF 方法計算兩個已完成分詞與去停用詞文本的餘弦相似度 (Cosine Similarity)。
    
    參數:
        text1_words: 第一個文本的分詞結果（字串形式，詞之間以空格分隔；或詞彙列表）
        text2_words: 第二個文本的分詞結果（字串形式，詞之間以空格分隔；或詞彙列表）
    
    回傳:
        float: 餘弦相似度值（範圍：0-1，1 表示方向完全相同/高度相似）
    """
    
    # 確保輸入為字串格式（若輸入為列表 list，則將其合併為以空格分隔的字串）
    if isinstance(text1_words, list):
        text1_words = ' '.join(text1_words)
    if isinstance(text2_words, list):
        text2_words = ' '.join(text2_words)
    
    # 初始化 TF-IDF 向量化工具
    # 這會將文本轉換為詞頻-逆文件頻率矩陣
    vectorizer = TfidfVectorizer()
    
    try:
        # 擬合數據並轉換文本為 TF-IDF 矩陣
        # 矩陣包含兩列向量，分別代表 text1 與 text2
        tfidf_matrix = vectorizer.fit_transform([text1_words, text2_words])
        
        # 計算兩個向量之間的餘弦相似度
        # cosine_similarity 預期輸入為矩陣，故使用切片 [0:1] 與 [1:2]
        similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]
        
        return similarity
    except ValueError:
        # 若文本為空或不包含任何可辨識的詞彙，TfidfVectorizer 可能會拋出錯誤
        return 0.0

In [ ]:
similarity12 = calculate_tfidf_cosine_similarity(text1, text2)
similarity13 = calculate_tfidf_cosine_similarity(text1, text3)

In [ ]:
print(f"專利1與專利2的相似度: {similarity12:.4f}\n專利1與專利3的相似度: {similarity13:.4f}")

## 2. 現代方法_大語言模型

In [ ]:
embedding_model = SentenceTransformer("./bge-base-zh-v1.5")

In [ ]:
embedding = embedding_model.encode([patent1, patent2, patent3])

In [ ]:
cos12 = embedding_model.similarity(embedding[0], embedding[1])
cos13 = embedding_model.similarity(embedding[0], embedding[2])

In [ ]:
print(f"專利1與專利2的相似度: {cos12.item():.4f}\n專利1與專利3的相似度: {cos13.item():.4f}")